In [2]:
import numpy as np
import pandas as pd
import os
#from lxml import etree as ET
from bs4 import BeautifulSoup
import re
import xml.etree.ElementTree as ET

In [3]:
df = pd.DataFrame()

In [4]:
def safe_find_text(soup, selector, attrs=None):
    finding = soup.find(selector, attrs=attrs)
    if finding:
        return finding.text
    else:
        return ""

Example of function usage:

In [18]:
text = safe_find_text


txt = open('Data 2018-2021/ECLI_NL_GHAMS_2019_194.xml',encoding='utf-8')
bs_obj = BeautifulSoup(txt, 'lxml')
full_text = bs_obj.text

safe_find_text(bs_obj,"inhoudsindicatie")

'\nCeltic-zaak: vervolging voetbalsupporters voor openlijke geweldpleging na ongeregeldheden voorafgaand aan Ajax-Celtic op 6 november 2013. Openbaar ministerie niet-ontvankelijk verklaard in vervolging Celtic-supporter op de voet van artikel 359a Sv wegens ernstige schending verbaliseringsplicht, die vastgesteld kon worden aan de hand van in een zeer laat stadium aan het dossier toegevoegde camerabeelden. Politie treft verwijt onjuiste verslaglegging ten aanzien van de kern van waar het in deze strafzaak om gaat. De met vervolging belaste ambtenaren treft het verwijt er onvoldoende zorg voor te hebben gedragen dat de door politieambtenaren vervaardigde camerabeelden tijdig aan het dossier konden worden toegevoegd.\n'

In [35]:
string = "hij in of omstreeks de periode van 4 november 2016 tot en met 6 november 2016 in arnhem en/of in nederland en/of in duitsland en/of in oostenrijk en/of in hongarije, opzettelijk een of meer minderjarige(n)"
crime_date_pattern = 'op of omstreeks [0-9]* [a-z]* [0-9]*|[op]*[in]* of omstreeks de periode van [0-9]* [a-z]*[ 0-9]* tot en met [0-9]* [a-z]* [0-9]*'

re.findall(crime_date_pattern, string)

['in of omstreeks de periode van 4 november 2016 tot en met 6 november 2016']

# Final loop

Patterns used to extract structure

In [ ]:
pattern_bewijs = '\*new_par\*[0-9]*bewijs[^\*]*'
pattern_tll = '\*new_par\*[0-9]*tenlastelegging[^\*]*'

pattern_tll_2 = '[\s*$]*[0-9]*te[n]*laste[n]*legging[\s*$]+[\s*$]+[^\*]*'
pattern_tll_bijlage = '[\s*$]*[0-9]*bijlage[:]*\s*[de\s]*tenlastelegging[\s*$]+[^\*]*'
tll_pattern_def = '[[\s*$]*[0-9]*[\s]*tenlastelegging[\s*$]+[\s*$]+[^\*]*]|[[\s*$]*[0-9]*bijlage[:]*\s*[de\s]*tenlastelegging[\s*$]+[^\*]*]'

pattern_bewijs_2 = '[\s*$]+[\s*$]+[0-9]*bewijs[^\*]*|[\s*$]+[\s*$]+[0-9]*bewijsmiddelen[^\*]*|[\s*$]+[\s*$]+[0-9]*het\sbewijs[^\*]*'

crime_date_pattern = 'op of omstreeks [0-9]* [a-z]* [0-9]*|[op]*[in]* of omstreeks de periode van [0-9]* [a-z]*[ 0-9]* tot en met [0-9]* [a-z]* [0-9]*'


Loop to read xml files downloaded from rechtspraak & filter out : documents >1000 characters and subject : strafrecht

Note: dcterms: is found in the xml files and we use it to filter for certain values as seen below


In [ ]:
path = src_folder


for filename in os.listdir(path):
    if not filename.endswith(".xml"): continue
    #fullname = os.path.join(path,filename)
    source = src_folder + filename
    txt = open(source,encoding='utf-8')
    bs_obj = BeautifulSoup(txt, 'lxml')
    
    destination = dst_folder + source
    full_text = bs_obj.text
    if len(str(full_text)) >= 1000:  #met lengte filter
        if "Strafrecht" in safe_find_text(bs_obj,"dcterms:subject"): #met strafrecht filter
            shutil.move(source,destination)


Loop to open filtered xml files (strafrecht + 1000char), and create dataframe using certain parts of the text containg the regular expression seen above

In [64]:
tll_list = [] # create empty lists for final dataframe
bv_list = []
df["text"] = ""
df["is_tll"] = 0
df["is_bewijs"] = ""
xml_list = [] 
ecli = []
subject = []
zaak_nr = []
spatial = []
procedure = []
wb_list= []
bijlage_list=[]
beslag_list = []
tll_2 = []
bewijs_2 = []
final_tll = []
final_bewijs = []
counter = 0

df_sections = pd.DataFrame()
path= "Dataset"

info_dict = dict()
info_list = []

cleaned_bewijs = []
cleaned_tll = []
crime_date = []

for filename in os.listdir(path):
    if not filename.endswith(".xml"): continue
    fullname = os.path.join(path,filename)
    txt = open(fullname,encoding='utf-8')
    bs_obj = BeautifulSoup(txt, 'lxml')
    full_text = bs_obj.text
    if len(str(full_text)) >= 1000:  #met lengte filter
        counter += 1 
        xml_list.append(bs_obj) 

        if "Strafrecht" in safe_find_text(bs_obj,"dcterms:subject"): #met strafrecht filter
            info_dict = dict()
            info_dict['filename'] = filename
            info_dict['ecli'] = safe_find_text(bs_obj,"dcterms:identifier")
            info_dict['subject'] = safe_find_text(bs_obj,"dcterms:subject")
            info_dict['spatial'] = safe_find_text(bs_obj,"dcterms:spatial")
            info_dict['case_nr'] = safe_find_text(bs_obj,"psi:zaaknummer")
            info_dict['date'] = safe_find_text(bs_obj, "dcterms:date")
            info_dict['inhoudsindicatie'] = safe_find_text(bs_obj, "inhoudsindicatie")
            
            #print('_________',filename,'________')
            #print(len(str(full_text)))
            text = bs_obj.findAll("section")
            section_text = [x.text for x in text]
            #print(section_text)
            for sub_section in section_text: # loop for reading bs object and find patterns
                match_tll = re.findall(pattern_tll_2, sub_section.lower())
                match_tll_bijlage = re.findall(pattern_tll_bijlage, sub_section.lower())
                match_tot = re.findall(tll_pattern_def, sub_section.lower())
                if str(match_tot) != '[]':
                    tll_2.append(match_tot)
    
                match_bewijs = re.findall(pattern_bewijs_2, sub_section.lower())
                if str(match_bewijs) != []:
                    bewijs_2.append(match_bewijs)
                    
                match_crime_date = re.findall(crime_date_pattern, sub_section.lower())
                if match_crime_date not in crime_date:
                     if match_crime_date != []:
                        crime_date.append(''.join(match_crime_date))
            for item_t in tll_2:
                #print(item_t)
                #print(type(item_t))
                #print(repr(item_t))
                for inside_t in item_t:
                    cleaned_t = inside_t.replace('\n', ' ')
                    cleaned_tll.append(cleaned_t)
                    
            for item_b in bewijs_2:
                for inside_b in item_b:
                    cleaned_b = inside_b.replace('\n', ' ')
                    cleaned_bewijs.append(cleaned_b)
            
            info_dict["bewijs"] = str(cleaned_bewijs)
            info_dict["tll"] = str(cleaned_tll)
            info_dict["crime_date"] = str(crime_date)
            info_list.append(info_dict)
            
            #print(repr(tll_2))
            info_dict= dict()
            tll_2 = []
            bewijs_2 = []
            cleaned_bewijs = []
            cleaned_tll = []
            crime_date = []
            #print('--------------------------------')
            #print()

In [ ]:
df = pd.DataFrame(info_list)
print(df.shape)
#df = df[df['tll'] != '[]']
df = df.drop_duplicates(subset = 'filename')
